# Hugging Face - Summarization in Japanese

This source code builds the fine-tuned model of [google/mt5-small](https://huggingface.co/google/mt5-small) for Japanese summarization.

For more background and details, see [this blog post](https://tsmatz.wordpress.com/2022/11/25/huggingface-japanese-summarization/).

*back to [index](https://github.com/tsmatz/huggingface-finetune-japanese/)*

## Install required packages

In order to install core components, see [Readme](https://github.com/tsmatz/huggingface-finetune-japanese/).<br>
Install additional packages for running this notebook as follows.

Install packages depending on T5 tokenizer.

In [ ]:
!pip install protobuf==3.20.3

Install packages depending on rouge evaluation.

In [ ]:
!pip install absl-py rouge_score nltk

Install other dependent packages.

In [ ]:
!pip install numpy

## Check device

Check whether GPU is available.

In [1]:
import torch

if torch.cuda.is_available():
    print("GPU is enabled.")
    print("device count: {}, current device: {}".format(torch.cuda.device_count(), torch.cuda.current_device()))
else:
    print("GPU is not enabled.")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

GPU is enabled.
device count: 1, current device: 0


## Prepare data

In this example, we use [XL-Sum Japanese dataset](https://huggingface.co/datasets/csebuetnlp/xlsum/viewer/japanese) in Hugging Face, which is the annotated article-summary pairs generated by BBC.<br>
This dataset has around 7000 samples for training.

In [2]:
from datasets import load_dataset

ds = load_dataset("csebuetnlp/xlsum", name="japanese")
ds

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset xlsum downloaded and prepared to /home/tsmatsuz/.cache/huggingface/datasets/csebuetnlp___xlsum/japanese/2.0.0/518ab0af76048660bcc2240ca6e8692a977c80e384ffb18fdddebaca6daebdce. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'summary', 'text'],
        num_rows: 7113
    })
    test: Dataset({
        features: ['id', 'url', 'title', 'summary', 'text'],
        num_rows: 889
    })
    validation: Dataset({
        features: ['id', 'url', 'title', 'summary', 'text'],
        num_rows: 889
    })
})

In [3]:
ds["train"][0]

{'id': '44789754',
 'url': 'https://www.bbc.com/japanese/44789754',
 'title': 'タイ洞窟から少年とコーチ、全員無事救出',
 'summary': 'タイ北部のタムルアン洞窟で10日夜、中に閉じ込められていた少年12人とサッカー・コーチの計13人のうち、最後の少年4人とコーチが水路を潜り無事脱出した。その約3時間後には、洞窟内で少年たちと留まっていた海軍ダイバー3人と医師も生還した。17日間も洞窟内にいた13人の救出に、タイ国内外で多くの人が安心し、喜んでいる。',
 'text': '救出作戦の間、洞窟内に少年たちと留まったタイ海軍のダイバーと医師も最後に無事脱出した。4人の写真は10日、タイ海軍特殊部隊がフェイスブックに掲載したもの タイ海軍特殊部隊はフェイスブックで、「これは奇跡なのか科学なのか、一体何なのかよくわからない。『イノシシ』13人は全員、洞窟から出た」と救助作戦の終了を報告した。「イノシシ」（タイ語で「ムーパ」）は少年たちの所属するサッカー・チームの愛称。 遠足に出かけた11歳から17歳の少年たちと25歳のサッカー・コーチは6月23日、大雨で増水した洞窟から出られなくなった。タイ内外から集まったダイバー約90人などが捜索に当たり、英国人ダイバー2人によって7月2日夜に発見された。地元のチェンライ県知事やタイ海軍特殊部隊が中心となった救助本部は当初、水が引くか、あるいは少年たちが潜水技術を習得するまで時間をかけて脱出させるつもりだったが、雨季による水位上昇と洞窟内の酸素低下の進行が懸念され、8日から3日連続の救出作戦が敢行された。 少年たちの脱出方法 ダイバーたちに前後を支えられ、水路内に張り巡らされたガイドロープをたどりながら、潜水経験のない少年たちは脱出した。8日に最初の4人、9日に4人、10日に残る5人が脱出し、ただちに近くのチェンライ市内の病院に搬送された。2週間以上洞窟に閉じ込められていたことを思えば、全員驚くほど心身ともに元気だという。 少年たちとコーチはレントゲンや血液検査などを受けた。少なくとも7日間は、経過観察のために入院を続けるという。 洞窟内の水を飲み、鳥やコウモリの排泄物に接触した可能性のある13人は、病原体に感染しているおそれがあるため隔離されてい

To generate inputs for fine-tuning, now I tokenize each text and convert into token ids.

First, load tokenizer in pre-trained ```google/mt5-small``` model.

In [4]:
from transformers import AutoTokenizer

t5_tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")

Downloading:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/553 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

/home/tsmatsuz/.local/lib/python3.8/site-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


For fine-tuning, apply tokenization for dataset.

In [5]:
def tokenize_sample_data(data):
    # Max token size is 14536 and 215 for inputs and labels, respectively.
    # Here I restrict these token size.
    input_feature = t5_tokenizer(data["text"], truncation=True, max_length=1024)
    label = t5_tokenizer(data["summary"], truncation=True, max_length=128)
    return {
        "input_ids": input_feature["input_ids"],
        "attention_mask": input_feature["attention_mask"],
        "labels": label["input_ids"],
    }

In [6]:
tokenized_ds = ds.map(
    tokenize_sample_data,
    remove_columns=["id", "url", "title", "summary", "text"],
    batched=True,
    batch_size=128)
tokenized_ds

  0%|          | 0/56 [00:00<?, ?ba/s]

  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/7 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 7113
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 889
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 889
    })
})

## Fine-tune

In this example, we use mT5 model.<br>
There exist several sizes of mT5 and I'll use small one (```google/mt5-small```) to fit to memory in my machine. The name is "small", but it's still so large.

In [7]:
from transformers import AutoConfig, AutoModelForSeq2SeqLM

# see https://huggingface.co/docs/transformers/main_classes/configuration
mt5_config = AutoConfig.from_pretrained(
    "google/mt5-small",
    max_length=128,
    length_penalty=0.6,
    no_repeat_ngram_size=2,
    num_beams=15,
)
model = (AutoModelForSeq2SeqLM
         .from_pretrained("google/mt5-small", config=mt5_config)
         .to(device))

Downloading:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

We prepare data collator, which works for preprocessing data.

For the sequence-to-sequence (seq2seq) task, we need to not only stack the inputs for encoder, but also prepare for the decoder side. In seq2seq setup, a common technique called "teach forcing" will then be applied in decoder.<br>
These tasks are not needed to manually setup in Hugging Face, and ```DataCollatorForSeq2Seq``` will take care of all steps.

In this collator, the padded token will also be filled with label id -100.<br>
This token will then be ignored in the sebsequent loss computation and evaluation.

In [8]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(
    t5_tokenizer,
    model=model,
    return_tensors="pt")

We also prepare metrics function for evaluation in the training.<br>
Measuring the quality of generated text is very difficult, and BLEU and ROUGE are often used.

Briefly speaking, BLEU measures how many of n-grams in the generated (predicted) text are overlaped in the reference text. This score is used for evaluation, especially in the machine translation task.
However, in summarization, we need all important words (which appears on the reference text) in the generated text. This is because we often use ROUGE in summarization tasks.
The idea of ROUGE is similar to BLEU, but it also measures how many of n-grams in the reference text appears in the generated (predicted) text. (This is why the name of ROUGE includes "RO", which means "Recall-Oriented".)<br>
There also exist variations, ROUGE-L and ROUGE-Lsum, which also measures the longest common substrings (LCS).

In Hugging Face, you don't need to manually implement these logics and can use built-in objects for scoring these matrics.<br>
In this example, I have configured mT5 tokenization as custom tokenization in computation (which is based on SentencePiece Unigram segmentation), because the white space tokenization is used as default in ROUGE evaluation.

> Note : You can also specify multilingual stemmer.

> Note : As I have mentioned above, the padded token id becomes -100 by data collator and I then also convert it into padded token id before processing.

In [9]:
import evaluate
import numpy as np
from nltk.tokenize import RegexpTokenizer

rouge_metric = evaluate.load("rouge")

def tokenize_sentence(arg):
    encoded_arg = t5_tokenizer(arg)
    return t5_tokenizer.convert_ids_to_tokens(encoded_arg.input_ids)

def metrics_func(eval_arg):
    preds, labels = eval_arg
    # Replace -100
    labels = np.where(labels != -100, labels, t5_tokenizer.pad_token_id)
    # Convert id tokens to text
    text_preds = t5_tokenizer.batch_decode(preds, skip_special_tokens=True)
    text_labels = t5_tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Add punctuation before sentence tokenization
    text_preds = [(p if p.endswith(("!", "！", "?", "？", "。")) else p + "。") for p in text_preds]
    text_labels = [(l if l.endswith(("!", "！", "?", "？", "。")) else l + "。") for l in text_labels]
    # Insert a line break (\n) in each sentence for ROUGE scoring
    sent_tokenizer_jp = RegexpTokenizer(u'[^!！?？。]*[!！?？。]')
    text_preds = ["\n".join(np.char.strip(sent_tokenizer_jp.tokenize(p))) for p in text_preds]
    text_labels = ["\n".join(np.char.strip(sent_tokenizer_jp.tokenize(l))) for l in text_labels]
    # compute ROUGE score with custom tokenization
    return rouge_metric.compute(
        predictions=text_preds,
        references=text_labels,
        tokenizer=tokenize_sentence
    )

Before fine-tuning, now I check ROUGE score with plain mT5 model. Here I check scores for top 5 rows in test dataset.

The score is very low, because this model is not trained for any downstream tasks. (It's just trained by unsupervised approach.)

> Note : In order to avoid suboptimal text generation, here I have applied beam search for the text generation algorithm.

In [10]:
from torch.utils.data import DataLoader

sample_dataloader = DataLoader(
    tokenized_ds["test"].with_format("torch"),
    collate_fn=data_collator,
    batch_size=5)
for batch in sample_dataloader:
    with torch.no_grad():
        preds = model.generate(
            batch["input_ids"].to(device),
            num_beams=15,
            num_return_sequences=1,
            no_repeat_ngram_size=1,
            remove_invalid_values=True,
            max_length=128,
        )
    labels = batch["labels"]
    break

metrics_func([preds, labels])

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'rouge1': 0.06507595140684969,
 'rouge2': 0.03374507990060038,
 'rougeL': 0.0650759514068497,
 'rougeLsum': 0.06448736750547615}

We prepare training arguments for fine-tuning.<br>
In this example, we use HuggingFace transformer trainer class, with which you can run training without manually writing training loop.

In usual training evaluation, training loss and accuracy will be computed and evaluated, by comparing the generated logits with labels. However, as we saw above, we want to evaluate ROUGE score using the predicted tokens.<br>
To simplify these sequence-to-sequence specific steps, here I use built-in ```Seq2SeqTrainingArguments``` and ```Seq2SeqTrainer``` classes in HuggingFace, instead of usual ```TrainingArguments``` and ```Trainer```.<br>
By setting ```predict_with_generate=True``` in this class, the predicted tokens generated by  ```model.generate()``` will be used in each evaluation.

The checkpoint files (in each 500 steps) are saved in the folder named ```mt5-summarize-ja```.

> Note : Do not use FP16 precision in mT5 fine-tuning.

> Note : In general, the saved checkpoints in the training will become so large.<br>
> Set ```save_total_limit``` property (which limits the total amount of checkpoints by deleting the older ones) to save disk spaces, or expand disks in Azure VM. (See [here](https://learn.microsoft.com/en-us/azure/virtual-machines/linux/expand-disks) to expand disks in Azure.)

In [11]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir = "mt5-summarize-ja",
    log_level = "error",
    num_train_epochs = 10,
    learning_rate = 5e-4,
    lr_scheduler_type = "linear",
    warmup_steps = 90,
    optim = "adafactor",
    weight_decay = 0.01,
    per_device_train_batch_size = 2,
    per_device_eval_batch_size = 1,
    gradient_accumulation_steps = 16,
    evaluation_strategy = "steps",
    eval_steps = 100,
    predict_with_generate=True,
    generation_max_length = 128,
    save_steps = 500,
    logging_steps = 10,
    push_to_hub = False
)

Build trainer. (Put it all together.)

Because the cost of evaluation computation (ROUGE scoring) is so high, I have then decreased the number of rows in validation set.

In [12]:
from transformers import Seq2SeqTrainer
trainer = Seq2SeqTrainer(
    model = model,
    args = training_args,
    data_collator = data_collator,
    compute_metrics = metrics_func,
    train_dataset = tokenized_ds["train"],
    eval_dataset = tokenized_ds["validation"].shard(num_shards=45, index=0),
    tokenizer = t5_tokenizer,
)

Now let's run training.<br>
As you will find, ROUGE scores are growing during training.

> Note : As I have mentioned above, make sure that you have enough disk space.

In [13]:
trainer.train()

Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
100,4.213300,3.428347,0.280613,0.132872,0.242478,0.249972
200,3.640700,3.059172,0.330911,0.172586,0.283457,0.291307
300,3.352300,2.855823,0.315189,0.159415,0.273967,0.284733
400,3.294700,2.776553,0.371320,0.208282,0.306402,0.323169
500,3.156700,2.724403,0.367068,0.195358,0.311227,0.319018
600,3.059600,2.652614,0.390743,0.226202,0.334307,0.344931
700,2.826800,2.645617,0.408129,0.235066,0.346507,0.362483
800,2.924800,2.579337,0.393283,0.213991,0.321050,0.337933
900,2.761400,2.572187,0.391245,0.219717,0.323080,0.336895
1000,2.754600,2.532084,0.400207,0.226026,0.339965,0.356957


TrainOutput(global_step=2220, training_loss=3.0295667949023546, metrics={'train_runtime': 18608.4195, 'train_samples_per_second': 3.822, 'train_steps_per_second': 0.119, 'total_flos': 6.575542928726016e+16, 'train_loss': 3.0295667949023546, 'epoch': 10.0})

In order to use it later, you can save the trained model.

In [14]:
import os

os.makedirs("./trained_for_summarization_jp", exist_ok=True)
if hasattr(trainer.model, "module"):
    trainer.model.module.save_pretrained("./trained_for_summarization_jp")
else:
    trainer.model.save_pretrained("./trained_for_summarization_jp")

Load pre-trained model from local.

In [6]:
from transformers import AutoModelForSeq2SeqLM

model = (AutoModelForSeq2SeqLM
         .from_pretrained("./trained_for_summarization_jp")
         .to(device))

## Generate Text (Summarize) with Fine-Tuned Model

Now let's see how it generates text for summarization with fine-tuned model.<br>
Here I generate the summarized text of test data, which has not seen in the training set.

> Note : The article in XL-Sum dataset is created by removing the first sentence (headline sentence) of BBC news source, and the first sentence is then used for summary.<br>
>  For this reason, there might exist several mismatch between article and summary in test data. (Choose appropriate samples for checking.)

In [9]:
from torch.utils.data import DataLoader

# Predict with test data (first 5 rows)
sample_dataloader = DataLoader(
    tokenized_ds["test"].with_format("torch"),
    collate_fn=data_collator,
    batch_size=5)
for batch in sample_dataloader:
    with torch.no_grad():
        preds = model.generate(
            batch["input_ids"].to(device),
            num_beams=15,
            num_return_sequences=1,
            no_repeat_ngram_size=1,
            remove_invalid_values=True,
            max_length=128,
        )
    labels = batch["labels"]
    break

# Replace -100 (see above)
labels = np.where(labels != -100, labels, t5_tokenizer.pad_token_id)

# Convert id tokens to text
text_preds = t5_tokenizer.batch_decode(preds, skip_special_tokens=True)
text_labels = t5_tokenizer.batch_decode(labels, skip_special_tokens=True)

# Show result
print("***** Input's Text *****")
print(ds["test"]["text"][0])
print("***** Summary Text (True Value) *****")
print(text_labels[0])
print("***** Summary Text (Generated Text) *****")
print(text_preds[0])

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


***** Input's Text *****
トム・エッジントン BBCリアリティー・チェック（ファクトチェック）チーム かつて労働党党首も務めたブレア氏は、BBCラジオ4の番組「Today」で、「議会は行き詰まった。議会が決められないなら、国民が決める形に戻ろう」と語った。 労働党の公式な立場は、テリーザ・メイ英首相が欧州連合（EU）と合意した離脱協定案が議会で否決された場合、解散総選挙の圧力をかけるというもの。もし総選挙が実現しなかった場合は、再度の国民投票を支持するのも選択肢になりうると、労働党は表明している。 しかしメイ首相は、再国民投票の予測を否定している。メイ氏は下院議員らに対し、2016年に実施した国民投票の結果が「尊重されるべきだ」と繰り返し語ってきた。 だが、もしブレア氏が求めている通り、下院がブレグジットをめぐる膠着（こうちゃく）状態を打ち破るために2度目の国民投票を実施すると決定したら、どうなるのだろうか？ 英選挙管理委員会はBBCニュースに対し、「適切な対応策」を有しており、「あらゆる予定外の投票に迅速に対応する」準備ができていると語った。 期限は迫っている イギリスのEU離脱予定日は、2019年3月29日。残り100日を切り、時間が最も差し迫った問題だ。 英議会が2度目の国民投票実施を採択した場合、投票規則や選挙運動規則を定める法律にも上下両院の支持が必要になる。 2016年の国民投票では、投票日の7カ月前に関連法案が議決された。 しかし、今回はもっと早い法制化が可能なのだろうか？ 法制化の速度を上げるため、前回の国民投票に関する諸規則を定めた2015年国民投票法をひな型にし、実質的に大部分を写してしまうのが、あり得る選択肢の1つだ。 英ユニヴァーシティー・コレッジ・ロンドン公共政策大学院憲法ユニットのアラン・レンウィック副ユニット長は、「理論上、このやり方は非常に素早く完了できる」と話す。 もしこのやり方が採用されても、法案の議会通過はおよそ11週間かかるとレンウィック氏は推計している。 この予定表を基にすると、法案通過は2月後半になると予想される。ただし、法制過程を今開始すればの話だ。 投票用紙の選択肢を、2016年の国民投票における「離脱」か「残留」かの2択ではなくし、複数の選択肢を含めるよう下院が要求した場合、かかる時間はもっと

In [12]:
print("***** Input's Text *****")
print(ds["test"]["text"][2])
print("***** Summary Text (True Value) *****")
print(text_labels[2])
print("***** Summary Text (Generated Text) *****")
print(text_preds[2])

***** Input's Text *****
このワクチンは複数の動物実験で、安全性や、効果的な免疫反応を引き起こすことが示されている。 今回の第1段階の後には、6000人を対象とした別の臨床試験が今年10月に予定されている。 インペリアル･コレッジ･ロンドンのチームは、2021年の早い時期からイギリスや海外でワクチンを配布できるようにしたいとしている。 ＜関連記事＞ 世界中では約120のワクチンの開発が進められている。英オックスフォード大学の専門家たちはすでに臨床試験を開始している。 新しいアプローチ 多くの従来のワクチンは、弱体化させたウイルスや改変したウイルスなどがもとになっている。しかし今回のワクチンは新しいアプローチに基づいたもので、遺伝子のRNA（リボ核酸）を使う。 筋肉に注射すると、RNAは自己増殖し、新型ウイルスの表面にみられるスパイクタンパク質のコピーをつくるよう、体内の細胞に指示を出す。 この方法で、COVID-19（新型ウイルスによる感染症）を発症することなく新型ウイルスを認識して戦うための免疫システムを訓練できるという。 シャトック教授は、「我々はゼロからワクチンを製造し、わずか数カ月で臨床試験に持ち込むことができた」と述べた。 「我々のアプローチがうまくいって、ワクチンがこの病気を効果的に防御できれば、将来的なアウトブレイク（大流行）への対応方法に革命をもたらす可能性がある」 主任研究員のカトリーナ・ポロック博士は、ワクチンの効果に期待している この研究の主任研究員、カトリーナ・ポロック博士は、「参加者に大きな免疫反応がみられるだろうと、慎重ながらも楽観的に感じられなかったら、私はこの臨床試験に取り組んでいなかっただろう」と付け加えた。 「前臨床データは非常に期待がもてるものだった。感染から保護しておきたい免疫反応である中和抗体応答は確認できているが、このワクチンを評価するにはまだ道のりは長い」 この研究は英政府から4100万ポンド（約54億5500万円）の資金提供を受けている。ほかにも500万ポンド（約6億6500万円）の寄付が寄せられている。 「ウイルスを倒すのに協力したくて志願」 金融業界で働くキャシーさん（39）は、インペリアル･コレッジ･ロンドンの臨床試験に参加している最初のボランティアの1人だ。 新型ウイルスとの戦いの一端を